In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 6.4e-07}
band = [3]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.017512                    1  0.024430
0.000750        2  0.019339                    2  0.025867
0.001052        3  0.023140                    3  0.028945
0.001476        4  0.027133                    4  0.032138
0.002070        5  0.031276                    5  0.035454
0.002904        6  0.035571                    6  0.038892
0.004074        7  0.040011                    7  0.042432
0.005714        8  0.044590                    8  0.046093
0.008015        9  0.049292                    9  0.049879
0.011243       10  0.054118                   10  0.053795
0.015771       11  0.059054                   11  0.057836
0.022122       12  0.064104                   12  0.062005
0.031031       13  0.069224                   13  0.066286
0.043528       14  0.073778                   14  0.070178
0.061057       15  0.077191                   15  0.073248
0.085645       16  0.080008                   16  0.075949
0.120136       17  0.082693                   17  0.078687
0.168516       18  0.085266                   18  0.081476
0.236378       19  0.087704                   19  0.084300
0.331549       20  0.090018                   20  0.087133
0.465100       21  0.091904                   21  0.089691
0.652400       22  0.087873                   22  0.087507
0.915100       23  0.076312                   23  0.079124
1.283650       24  0.063337                   24  0.069247
1.800600       25  0.051626                   25  0.059790
2.525700       26  0.041308                   26  0.050671
3.542800       27  0.032408                   27  0.041884
4.969550       28  0.025829                   28  0.034335
6.970850       29  0.021927                   29  0.028517
9.778100       30  0.019195                   30  0.023442
13.715850      31  0.016556                   31  0.018446
19.239350      32  0.013930                   32  0.013820
26.987250      33  0.012489                   33  0.010773
37.855300      34  0.012293                   34  0.009439
53.100050      35  0.011924                   35  0.008680
73.887500      36  0.011149                   36  0.008194
97.662500      37  0.010106                   37  0.007767
121.437500     38  0.008666                   38  0.007012
145.212500     39  0.007071                   39  0.006029
168.987500     40  0.005660                   40  0.005077
192.762500     41  0.004364                   41  0.004134
216.537500     42  0.003126                   42  0.003178
240.312500     43  0.001891                   43  0.002200
264.087500     44  0.000604                   44  0.001195
287.862500     45 -0.000621                   45  0.000316
311.637500     46 -0.000583                   46  0.000540
335.412500     47  0.001264                   47  0.002302
359.187500     48  0.003709                   48  0.004655
382.962500     49  0.006026                   49  0.006980
406.737500     50  0.008217                   50  0.009236
430.512500     51  0.010316                   51  0.011427
454.287500     52  0.012342                   52  0.013555
478.062500     53  0.014305                   53  0.015630
501.837500     54  0.016206                   54  0.017642
525.612500     55  0.018041                   55  0.019574
549.387500     56  0.019800                   56  0.021429
573.162500     57  0.021469                   57  0.023192
596.937500     58  0.023036                   58  0.024851
620.712500     59  0.024484                   59  0.026389
644.487500     60  0.025807                   60  0.027802
668.262500     61  0.027001                   61  0.029083
692.037500     62  0.028077                   62  0.030246
715.812500     63  0.029052                   63  0.031303
739.587500     64  0.029946                   64  0.032271
763.362500     65  0.030779                   65  0.033167
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -22.696863  0.000000 -22.696863 -22.687126  4.466527e-08   
0.000624    2     -22.696864  0.000003 -22.696862 -22.687127  2.297030e-06   
0.000876    3     -22.696865  0.000004 -22.696861 -22.687127  3.227479e-06   
0.001229    4     -22.696866  0.000006 -22.696860 -22.687127  4.597625e-06   
0.001723    5     -22.696866  0.000008 -22.696859 -22.687127  6.614018e-06   
0.002417    6     -22.696867  0.000012 -22.696856 -22.687127  9.579745e-06   
0.003391    7     -22.696869  0.000017 -22.696852 -22.687127  1.394013e-05   
0.004757    8     -22.696870  0.000025 -22.696845 -22.687127  2.034617e-05   
0.006672    9     -22.696871  0.000036 -22.696835 -22.687126  2.975169e-05   
0.009359    10    -22.696872  0.000053 -22.696820 -22.687123  4.355408e-05   
0.013128    11    -22.696873  0.000077 -22.696795 -22.687120  6.379877e-05   
0.018415    12    -22.696872  0.000113 -22.696758 -22.687113  9.347533e-05   
0.025830    13    -22.696868  0.000166 -22.696702 -22.687102  1.369514e-04   
0.036232    14    -22.696859  0.000242 -22.696617 -22.687084  2.005977e-04   
0.050823    15    -22.696843  0.000354 -22.696489 -22.687055  2.932635e-04   
0.071291    16    -22.696815  0.000513 -22.696302 -22.687012  4.269839e-04   
0.100000    17    -22.696770  0.000740 -22.696030 -22.686945  6.191433e-04   
0.140271    18    -22.696699  0.001063 -22.695636 -22.686846  8.951850e-04   
0.196760    19    -22.696585  0.001520 -22.695065 -22.686697  1.291617e-03   
0.275997    20    -22.696407  0.002165 -22.694242 -22.686475  1.860626e-03   
0.387100    21    -22.696127  0.003070 -22.693057 -22.686144  2.676252e-03   
0.543100    22    -22.695690  0.004332 -22.691359 -22.685653  3.843236e-03   
0.761700    23    -22.695084  0.006001 -22.689083 -22.684989  5.445684e-03   
1.068500    24    -22.694368  0.008058 -22.686310 -22.684200  7.532743e-03   
1.498800    25    -22.693589  0.010507 -22.683082 -22.683329  1.019140e-02   
2.102400    26    -22.692781  0.013391 -22.679390 -22.682417  1.355493e-02   
2.949000    27    -22.692014  0.016767 -22.675248 -22.681548  1.776827e-02   
4.136600    28    -22.691428  0.020739 -22.670688 -22.680872  2.298439e-02   
5.802500    29    -22.691175  0.025583 -22.665591 -22.680550  2.943906e-02   
8.139200    30    -22.691374  0.031852 -22.659522 -22.680730  3.751294e-02   
11.417000   31    -22.692301  0.040232 -22.652069 -22.681714  4.760000e-02   
16.014700   32    -22.694566  0.051514 -22.643052 -22.684120  6.005380e-02   
22.464000   33    -22.699172  0.066763 -22.632410 -22.688899  7.539201e-02   
31.510500   34    -22.707025  0.087999 -22.619027 -22.696977  9.501590e-02   
44.200100   35    -22.718848  0.118300 -22.600548 -22.709309  1.215369e-01   
62.000000   36    -22.736319  0.160913 -22.575405 -22.727879  1.584105e-01   
85.775000   37    -22.760994  0.216988 -22.544005 -22.754529  2.081390e-01   
109.550000  38    -22.787206  0.271663 -22.515543 -22.783101  2.585885e-01   
133.325000  39    -22.815552  0.324417 -22.491136 -22.813979  3.092171e-01   
157.100000  40    -22.846452  0.375232 -22.471220 -22.847443  3.596618e-01   
180.875000  41    -22.879857  0.424577 -22.455280 -22.883423  4.099432e-01   
204.650000  42    -22.915761  0.472772 -22.442989 -22.921894  4.600590e-01   
228.425000  43    -22.954228  0.520044 -22.434184 -22.962868  5.099828e-01   
252.200000  44    -22.995412  0.566555 -22.428858 -23.006381  5.596925e-01   
275.975000  45    -23.039593  0.612435 -22.427158 -23.052478  6.091562e-01   
299.750000  46    -23.086967  0.658060 -22.428907 -23.101005  6.585745e-01   
323.525000  47    -23.135971  0.705422 -22.430549 -23.150416  7.095073e-01   
347.300000  48    -23.183932  0.756942 -22.426990 -23.198429  7.640052e-01   
371.075000  49    -23.229626  0.813081 -22.4165

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')